# Polars benchmark alternatives

In [1]:
import polars as pl

In [ ]:
%%time
# This query errors out on a Macbook M1 with 64GB of RAM
pl.read_csv("~/data/G1_1e9_1e2_0_0.csv").filter(pl.col("id1") < "id016").group_by(
    ["id1", "id2"]
).agg(pl.sum("v1").alias("v1_sum")).collect()

In [5]:
%%time
pl.scan_csv("~/data/G1_1e9_1e2_0_0.csv").filter(pl.col("id1") < "id016").group_by(
    ["id1", "id2"]
).agg(pl.sum("v1").alias("v1_sum")).collect()

CPU times: user 1min 35s, sys: 1min 11s, total: 2min 47s
Wall time: 56.2 s


id1,id2,v1_sum
str,str,i64
"""id015""","""id043""",298635
"""id003""","""id029""",300119
"""id009""","""id049""",300354
"""id007""","""id086""",300120
"""id015""","""id044""",300277
"""id002""","""id010""",298593
"""id011""","""id099""",302014
"""id003""","""id097""",298624
"""id012""","""id051""",300107


In [2]:
%%time
pl.scan_parquet("~/data/G1_1e9_1e2_0_0.parquet").filter(
    pl.col("id1") < "id016"
).group_by(["id1", "id2"]).agg(pl.sum("v1").alias("v1_sum")).collect()

CPU times: user 39.4 s, sys: 6.06 s, total: 45.4 s
Wall time: 8.32 s


id1,id2,v1_sum
str,str,i64
"""id002""","""id080""",300074
"""id004""","""id096""",300097
"""id013""","""id019""",300946
"""id006""","""id027""",299293
"""id007""","""id043""",298637
"""id007""","""id061""",300144
"""id006""","""id023""",302003
"""id010""","""id041""",301516
"""id015""","""id044""",300277


## Delta table version=1 is just compacted

In [4]:
%%time
pl.scan_delta("~/data/deltalake/G1_1e9_1e2_0_0", version=1).filter(
    pl.col("id1") < "id016"
).group_by(["id1", "id2"]).agg(pl.sum("v1").alias("v1_sum")).collect()

CPU times: user 43 s, sys: 6.55 s, total: 49.6 s
Wall time: 7.24 s


id1,id2,v1_sum
str,str,i64
"""id011""","""id045""",301091
"""id015""","""id032""",298705
"""id002""","""id041""",301317
"""id002""","""id056""",299695
"""id007""","""id067""",300461
"""id007""","""id020""",300966
"""id004""","""id085""",300749
"""id006""","""id011""",301073
"""id006""","""id058""",301437


## Delta table version=2 is Z Ordered by id1

In [6]:
%%time
pl.scan_delta("~/data/deltalake/G1_1e9_1e2_0_0", version=2).filter(
    pl.col("id1") < "id016"
).group_by(["id1", "id2"]).agg(pl.sum("v1").alias("v1_sum")).collect()

CPU times: user 13.4 s, sys: 4.23 s, total: 17.6 s
Wall time: 3.5 s


id1,id2,v1_sum
str,str,i64
"""id005""","""id070""",299379
"""id001""","""id061""",301157
"""id012""","""id060""",300968
"""id007""","""id081""",298421
"""id010""","""id029""",301469
"""id008""","""id063""",298568
"""id011""","""id045""",301091
"""id013""","""id006""",300422
"""id015""","""id097""",301027
